In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import myFunctions2 as my
%load_ext autoreload
%autoreload 2

## Load the training data 
into feature matrix X, class labels y, and event ids:

In [ ]:
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, X, ids = my.load_csv_data(DATA_TRAIN_PATH)

## Data cleaning
Here we deal with problematic values.

In [ ]:
-

In [ ]:
# You can work with less samples, it saves time.
#print(tX.shape)
#N = 100000
#tX = tX[0:N]
#y = y[0:N]
#print(tX.shape)


# indices of each class
#idx_red = np.where(y == 1)[0]
#idx_blue = np.where(y == -1)[0]

#for d in range(1,tX.shape[1]):
#    f, axarr = plt.subplots(2, sharex=True)
#    axarr[0].hist(tX[idx_red,d], 50, normed=1, facecolor='red', alpha=0.5)
#    axarr[1].hist(tX[idx_blue,d], 50, normed=1, facecolor='blue', alpha=0.5)
#    plt.show()


# look for outliers (> 40*std)
#outliers = np.where(tX > 40)
# here we see that our single outlier is in fact a higgs boson event...
# should we keep it ???
#print(outliers[0], y[outliers[0]])
#plt.boxplot(tX)
#np.sum(y == 1)/len(y), np.sum(y == -1)/len(y)


#Is there samples that are identical ?
#ncols = tX.shape[1]
#dtype = tX.dtype.descr * ncols
#struct = tX.view(dtype)

#uniq, idx = np.unique(struct, return_index=True)
#tX = uniq.view(tX.dtype).reshape(-1, ncols)

#print(tX.shape)

# Method 4 : Ridge regression

In [ ]:
def cross_validation_RR(y, x, k_indices, k, lamb):
    """return the loss of ridge regression."""
    
    ## get k'th subgroup in test, others in train
    tx_tr = x[np.delete(k_indices, (k), axis=0).flatten()]
    y_tr = y[np.delete(k_indices, (k), axis=0).flatten()]
    tx_te = x[k_indices[k]]
    y_te = y[k_indices[k]]
    
    ## regression method
    
    #w = my.least_squares(y_tr, tx_tr)
    w = my.ridge_regression(y_tr, tx_tr, lamb)
    
    ## calculate the loss for train and test data :
    ## - with mse, mae or rmse
    #loss_tr = my.compute_loss(y_tr, tx_tr, w)
    #loss_te = my.compute_loss(y_te, tx_te, w)
    ## - with missclassification rates
    loss_tr, loss_te = my.compute_classerror(w, tx_tr, tx_te, y_tr, y_te)
    
       
    return loss_tr, loss_te

def cross_validation_RR_demo(y, X):
    ## parameters
    seed = 22
    k_fold = 10
    
    ## hyperparameters
    degrees = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    lambdas = np.logspace(-4, 1, 18)
    
    # split data in k fold
    k_indices = my.build_k_indices(y, k_fold, seed)
    
    # define lists to store the loss of training data and test data
    rmse_tr = np.zeros((len(degrees),k_fold))
    rmse_te = np.zeros((len(degrees),k_fold))
    best_lambda = np.zeros((len(degrees),))
    
    # Hyperparameter 1
    for d, degree in enumerate(degrees):
        print(degree)
        # Build polynomial function
        tX = my.build_poly(X, degree)
        
        loss_tr = np.zeros((len(lambdas),k_fold))
        loss_te = np.zeros((len(lambdas),k_fold))
        
        for l, lamb in enumerate(lambdas):
            # Cross-validation
            for k in range(k_fold):
                ## Store losses
                loss_tr[l,k], loss_te[l,k] = cross_validation_RR(y, tX, k_indices, k, lamb)
        ## Select automatically the value of lambda that minimize the missclassification rate       
        best_idx = np.argmin(np.mean(loss_te,axis=0))
        best_lambda[d] = lambdas[best_idx]
        
        ## Store losses
        rmse_tr[d,:] = loss_tr[best_idx,:]
        rmse_te[d,:] = loss_te[best_idx,:]
    
    ## Bias-Variance graphic
    my.bias_variance_decomposition_visualization(degrees, rmse_tr.T, rmse_te.T)   
    print(best_lambda)
    
    return rmse_tr, rmse_te, degrees, best_lambda

rmse_tr, rmse_te, degrees, best_lambda = cross_validation_RR_demo(y, tX)

## Make your predictions
First, train on the all datatset

In [ ]:
# Select the hyperparameter that have been chosen
final_degree = 10
final_lambda = 0.00239503
#10 0.00239503

# Build the polynomial basis, perform ridge regression
final_X = my.build_poly(tX, final_degree)
final_weights = my.ridge_regression(y, final_X, final_lambda)

# Take a look at the train error
y_pred = my.predict_labels(final_weights, final_X)
loss = len(np.nonzero(y_pred-y)[0])/len(y)
loss

In [ ]:
# Download test data
DATA_TEST_PATH = 'test.csv'
_, X_test, ids_test = my.load_csv_data(DATA_TEST_PATH)

In [ ]:
# Transform test data as it has been done for training : cleaning and polynomial basis
tX_test, mean_xtest, std_xtest = clean_data(X_test)
final_X_test = my.build_poly(tX_test, final_degree)

# Aplly the model to the test data in order to get predictions
OUTPUT_PATH = 'results.csv'
y_pred = my.predict_labels(final_weights, final_X_test)
# create submission file
my.create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

## Method 5 : Logistic regression

In [ ]:
def logistic_cross_validation(y, x, k_indices, k):
    """return the loss of ridge regression."""
    gamma = 0.01
    w = np.zeros((x.shape[1],1))
    batch_size = 5
    max_epochs = 300
    
    # get k'th subgroup in test, others in train: TODO
    tx_tr = x[np.delete(k_indices, (k), axis=0).flatten()]
    y_tr = y[np.delete(k_indices, (k), axis=0).flatten()]
    tx_te = x[k_indices[k]]
    y_te = y[k_indices[k]]
    
    # Stochastics gradient descent
    losses, ws = my.logistic_regression(y_tr, tx_tr, w, batch_size, max_epochs, gamma)
    best_idx = np.argmin(losses)
    w = ws[best_idx]
    
    loss_tr, loss_te = my.compute_logic_classerror(w, tx_tr, tx_te, y_tr, y_te)
        
    return loss_tr, loss_te



def logistic_demo(y, tx):
    
    y[np.where(y == -1)[0]] = 0
    seed = 22
    
    # prepare k-fold cross-validation
    k_fold = 10
    k_indices = my.build_k_indices(y, k_fold, seed)
    
    # losses matrices
    loss_tr = np.zeros((k_fold,))
    loss_te = np.zeros((k_fold,))
    
    #tx = my.build_poly(tx,2)
    
    # cross-validation
    for k in range(k_fold):
        loss_tr[k], loss_te[k] = logistic_cross_validation(y, tx, k_indices, k)
    
    return loss_tr, loss_te


loss_tr, loss_te = logistic_demo(y, tX)
print(np.mean(loss_tr), np.mean(loss_te))

## Method 6 : Logistic penalized regression

In [ ]:
def logic_cross_validation(y, x, k_indices, k, lambda_):
    """return the loss of ridge regression."""
    gamma = 0.01
    w = np.zeros((x.shape[1],1))
    batch_size = 5
    max_epochs = 300
    
    # get k'th subgroup in test, others in train: TODO
    tx_tr = x[np.delete(k_indices, (k), axis=0).flatten()]
    y_tr = y[np.delete(k_indices, (k), axis=0).flatten()]
    tx_te = x[k_indices[k]]
    y_te = y[k_indices[k]]
    
    # Stochastics gradient descent
    losses, ws = my.reg_logistic_regression(y_tr, tx_tr, w, batch_size, max_epochs, gamma, lambda_)
    best_idx = np.argmin(losses)
    w = ws[best_idx]
    
    loss_tr, loss_te = my.compute_logic_classerror(w, tx_tr, tx_te, y_tr, y_te)
        
    return loss_tr, loss_te



def reg_logistic_regression(y, tx):
    
    y[np.where(y == -1)[0]] = 0
    lambdas = np.logspace(-5,-1,10)
    
    seed = 22
    
    # prepare k-fold cross-validation
    k_fold = 10
    k_indices = my.build_k_indices(y, k_fold, seed)
    
    #tx = my.build_poly(tx,2)
    
    # losses matrices
    loss_tr = np.zeros((k_fold,len(lambdas)))
    loss_te = np.zeros((k_fold,len(lambdas)))
    
    for l, lambda_ in enumerate(lambdas):
        
        # cross-validation
        for k in range(k_fold):
            loss_tr[k,l], loss_te[k,l] = logic_cross_validation(y, tx, k_indices, k, lambda_)
            
    mse_tr = np.mean(loss_tr, axis=0)
    mse_te = np.mean(loss_te, axis=0)
    my.cross_validation_visualization(lambdas, mse_tr, mse_te)   
    
    return loss_tr, loss_te

loss_tr, loss_te = reg_logistic_regression(y, tX)

In [ ]:
gamma = 0.01
w = np.zeros((tX.shape[1],1))
batch_size = 5
max_epochs = 300
lambda_ = np.logspace(-5,-1,10)[7]
y[np.where(y == -1)[0]] = 0

losses, ws = my.reg_logistic_regression(y, tX, w, batch_size, max_epochs, gamma, lambda_)
best_idx = np.argmin(losses)
w = ws[best_idx]
print(losses[best_idx])
    

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, X_test, ids_test = my.load_csv_data(DATA_TEST_PATH)

In [ ]:
tX_test, mean_xtest, std_xtest = clean_data(X_test)
#final_X_test = my.build_poly(tX_test, degree)

OUTPUT_PATH = 'results.csv' # TODO: fill in desired name of output file for submission
y_pred = my.predict_logic_labels(w, tX_test)
y_pred[np.where(y_pred == 0)[0]] = -1
my.create_csv_submission(ids_test, y_pred, OUTPUT_PATH)